In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
"""Class implementing options pricing based on Monte-Carlo simulations.
Products priced: call, put, asain, barrier and knockout options."""

class OptionsPricing:
    """
    S0 = stock price at time 0
    vol = volatility of the instrument
    rf = risk free rate
    T = maturity
    N = number of simulations
    steps = number of steps"""
    
    def __init__(self, S0, vol, rf, T, N, steps = 250):
        self.S0 = S0
        self.vol = vol
        self.rf = rf
        self.T = T
        self.N = N
        self.M = steps
        
    @property 
    
    def S0(self):
        return self.__S0
    
    @property
    
    def vol(self):
        return self.__vol
    
    @property 
    
    def rf(self):
        return self.__rf
    
    @property
    
    def T(self):
        return self.__T
    
    @property
    
    def N(self):
        return self.__N
    
    @property
    
    def M(self):
        return self.__M
    
    @S0.setter 
    
    def S0(self, S0):
        self.__S0 = S0
    
    @vol.setter
    
    def vol(self, vol):
        self.__vol = vol
    
    @rf.setter 
    
    def rf(self, rf):
        self.__rf = rf 
    
    @T.setter
    
    def T(self, T):
        self.__T = T
    
    @N.setter
    
    def N(self, N):
        self.__N = N
        
    @M.setter
    
    def M(self, steps):
        self.__M = steps
        
    """Matrix of storing paths"""
    
    def paths(self):
        matrixPaths = np.zeros((self.N, self.M))
        matrixPaths[:,0] = self.S0
        for i in range(1,self.M):
            dt = self.T/self.M
            np.random.seed(123)
            Wt = self.vol*np.sqrt(dt)*np.random.normal(size=self.N)
            helper = matrixPaths[:,i-1]*np.exp((self.rf-self.vol**2/2)*dt+Wt)
            matrixPaths[:,i] = helper
        return matrixPaths
    
    #"""Plot of stock prices"""
    
    #def plotPaths(self):
        #paths = self.paths()
        #paths = pd.DataFrame(paths)
        #helper = []
        #for i in range(self.N):
            #helper.append(paths.iloc[i,:])
        #pd.concat(helper, axis=1).plot(figsize=(10,7))
        #plt.grid()
        #plt.title("Stock price paths.")
        #plt.show()
        
    """Price of European call option"""
    
    def callEU(self, strike):
        paths = self.paths()
        payoff = [max((S-strike),0) for S in paths[:,-1]]
        price = sum(payoff)/self.N
        priceDiscounted = price*np.exp(-self.rf*(self.T))
        return priceDiscounted
    
    """Price of European put option"""
    
    def putEU(self, strike):
        paths = self.paths()
        payoff = [max((strike-S),0) for S in paths[:,-1]]
        price = sum(payoff)/self.N
        priceDiscounted = price*np.exp(-self.rf*(self.T))
        return priceDiscounted
    
    """Price of Asian call option"""
    
    def callAsian(self, strike):
        paths = self.paths()
        payoff = [max((np.mean(paths[i,:])-strike),0) for i in range(self.N)]
        price = sum(payoff)/self.N
        priceDiscounted = price*np.exp(-self.rf*(self.T))
        return priceDiscounted
    
    """Price of Asian put option"""
    
    def putAsian(self, strike):
        paths = self.paths()
        payoff = [max((strike-np.mean(paths[i,:])),0) for i in range(self.N)]
        price = sum(payoff)/self.N
        priceDiscounted = price*np.exp(-self.rf*(self.T))
        return priceDiscounted
    
    """Price of Up-and-out Knockout call option"""
    
    def callUpAndOutKnock(self, strike, barrier):
        paths = self.paths()
        pathsReachedBarrier = paths > barrier
        payoff = []
        for i in range(self.N):
            if True not in pathsReachedBarrier[i,:]:
                payoff.append(max((paths[i,-1]-strike),0))
        price = sum(payoff)/self.N
        priceDiscounted = price*np.exp(-self.rf*(self.T))
        return priceDiscounted
    
    """Price of Up-and-in Knockout call option"""
    
    def callUpAndInKnock(self, strike, barrier):
        paths = self.paths()
        pathsReachedBarrier = paths > barrier
        payoff = []
        for i in range(self.N):
            if False not in pathsReachedBarrier[i,:]:
                payoff.append(max((paths[i,-1]-strike),0))
        price = sum(payoff)/self.N
        priceDiscounted = price*np.exp(-self.rf*(self.T))
        return priceDiscounted
    
    """Price of Down-and-out Knockout call option"""
    
    def callDownAndOutKnock(self, strike, barrier):
        paths = self.paths()
        pathsReachedBarrier = paths < barrier
        payoff = []
        for i in range(self.N):
            if True not in pathsReachedBarrier[i,:]:
                payoff.append(max((paths[i,-1]-strike),0))
        price = sum(payoff)/self.N
        priceDiscounted = price*np.exp(-self.rf*(self.T))
        return priceDiscounted
    
    """Price of Down-and-in Knockout call option"""
    
    def callDownAndInKnock(self, strike, barrier):
        paths = self.paths()
        pathsReachedBarrier = paths < barrier
        payoff = []
        for i in range(self.N):
            if False not in pathsReachedBarrier[i,:]:
                payoff.append(max((paths[i,-1]-strike),0))
        price = sum(payoff)/self.N
        priceDiscounted = price*np.exp(-self.rf*(self.T))
        return priceDiscounted
    
    
    """Price of Up-and-out Knockout put option"""
    
    def putUpAndOutKnock(self, strike, barrier):
        paths = self.paths()
        pathsReachedBarrier = paths > barrier
        payoff = []
        for i in range(self.N):
            if True not in pathsReachedBarrier[i,:]:
                payoff.append(max((strike-paths[i,-1]),0))
        price = sum(payoff)/self.N
        priceDiscounted = price*np.exp(-self.rf*(self.T))
        return priceDiscounted   
    
    """Price of Up-and-in Knockout put option"""
    
    def putUpAndInKnock(self, strike, barrier):
        paths = self.paths()
        pathsReachedBarrier = paths > barrier
        payoff = []
        for i in range(self.N):
            if False not in pathsReachedBarrier[i,:]:
                payoff.append(max((strike-paths[i,-1]),0))
        price = sum(payoff)/self.N
        priceDiscounted = price*np.exp(-self.rf*(self.T))
        return priceDiscounted
    
    
    
    """Price of Down-and-out Knockout put option"""
    
    def putDownAndOutKnock(self, strike, barrier):
        paths = self.paths()
        pathsReachedBarrier = paths < barrier
        payoff = []
        for i in range(self.N):
            if True not in pathsReachedBarrier[i,:]:
                payoff.append(max((strike-paths[i,-1]),0))
        price = sum(payoff)/self.N
        priceDiscounted = price*np.exp(-self.rf*(self.T))
        return priceDiscounted   
    
    """Price of Down-and-in Knockout put option"""
    
    def putDownAndInKnock(self, strike, barrier):
        paths = self.paths()
        pathsReachedBarrier = paths < barrier
        payoff = []
        for i in range(self.N):
            if False not in pathsReachedBarrier[i,:]:
                payoff.append(max((strike-paths[i,-1]),0))
        price = sum(payoff)/self.N
        priceDiscounted = price*np.exp(-self.rf*(self.T))
        return priceDiscounted   